<a href="https://colab.research.google.com/github/Muhammedafnanpk/Deep-learning-using-pytorch/blob/main/iris_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Model(nn.Module):
  def __init__(self,in_features=4,h1=8,h2=9,out_features=3):
    super().__init__()
    self.fc1=nn.Linear(in_features,h1)
    self.fc2=nn.Linear(h1,h2)
    self.out=nn.Linear(h2,out_features)

  def forward(self,x):
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.out(x)
    return x



In [4]:
torch.manual_seed(41)
model=Model()

In [34]:
import pandas as pd
url="https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
my_df=pd.read_csv(url)
my_df


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [36]:
my_df["species"]=my_df["species"].replace('setosa',0.0)
my_df["species"]=my_df["species"].replace('versicolor',1.0)
my_df["species"]=my_df["species"].replace('virginica',2.0)
my_df

/tmp/ipython-input-1549750296.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df["species"]=my_df["species"].replace('virginica',2.0)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [38]:
x=my_df.drop('species',axis=1)
y=my_df['species']

After mounting your Google Drive, you can load your data. Replace `'your_file_path_here.csv'` with the actual path to your file in Google Drive.

In [40]:
  X=x.values
  Y=y.values

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=41)

In [43]:
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
Y_train=torch.LongTensor(Y_train)
Y_test=torch.LongTensor(Y_test)

In [44]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [49]:
epochs=100
losses=[]
for i in range(epochs):
  y_pred=model.forward(X_train)
  loss=criterion(y_pred,Y_train)
  losses.append(loss.detach().numpy)

  if i%10==0:
    print((f'epoch={i},loss={loss}'))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

epoch=0,loss=0.05286576226353645
epoch=10,loss=0.04508381709456444
epoch=20,loss=0.03979949280619621
epoch=30,loss=0.035970740020275116
epoch=40,loss=0.03293721750378609
epoch=50,loss=0.03041938506066799
epoch=60,loss=0.026795217767357826
epoch=70,loss=0.02338254824280739
epoch=80,loss=0.020531881600618362
epoch=90,loss=0.018495969474315643


In [50]:
with torch.no_grad():
  y_eval=model.forward(X_test)
  loss=criterion(y_eval,Y_test)

In [51]:
loss

tensor(0.1816)

In [54]:
correct=0
with torch.no_grad():
  for i,data in enumerate(X_test):
    y_val=model.forward(data)
    print(f'{i+1:2}. {str(y_val):38} {Y_test[i]}')
  if y_val.argmax().item()==Y_test[i]:
    correct+=1
print(f'{correct} out of {len(Y_test)} = {100*correct/len(Y_test)}% correct')

 1. tensor([-7.1090,  3.8945,  9.0009])    2
 2. tensor([-9.7568,  1.8037, 15.7614])    2
 3. tensor([-10.9675,   2.8777,  16.6036]) 2
 4. tensor([-3.7644,  8.0391, -1.2691])    1
 5. tensor([-9.0332,  3.5238, 12.6383])    2
 6. tensor([-1.8244,  8.6935, -5.3425])    1
 7. tensor([-6.8734,  4.9821,  7.3703])    2
 8. tensor([-3.6489,  8.2259, -1.6743])    1
 9. tensor([-7.9281,  4.2588,  9.9576])    2
10. tensor([-10.4298,   1.8741,  16.8126]) 2
11. tensor([-6.4726,  5.1594,  6.5086])    2
12. tensor([ 12.4419,   2.4570, -21.9049]) 0
13. tensor([ 11.4217,   2.2044, -19.9136]) 0
14. tensor([-0.1940,  6.9626, -6.0924])    1
15. tensor([ 10.1568,   3.3059, -19.1088]) 0
16. tensor([-5.9767,  5.7989,  4.9675])    2
17. tensor([ 11.2091,   2.5254, -19.9652]) 0
18. tensor([-6.8898,  4.2924,  8.1948])    1
19. tensor([ 12.0366,   2.3664, -21.1260]) 0
20. tensor([  9.6811,   2.7818, -17.7338]) 0
21. tensor([-0.8226,  7.6358, -5.8090])    1
22. tensor([-10.0604,   2.6576,  15.3368]) 2
23. tensor

In [55]:
torch.save(model.state_dict(),'iris.pt')

In [56]:
new_model=Model()
new_model.load_state_dict(torch.load('iris.pt'))

<All keys matched successfully>

In [57]:
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)